In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import dask
import dask.array as da
import xarray as xr
import csv
from scipy.spatial import KDTree
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timedelta
import os

In [2]:
''' Corners of the box of interest '''
location = 10
centers  = [[-153.65, 66.70], # Alatna (2, 3572)
            [-155.69, 66.01], # Huslia (2, 3612)
            [-157.01, 66.96], # (2, 3548)
            [-157.82, 65.81], # (2, 3630)
            [-158.34, 65.23], # (2, 3674)
            [-153.05, 67.12], # (2, 3537)
            [-152.30, 67.12], # (2, 3539)
            [-150.92, 66.71], # (2, 3568)
            [-146.44, 66.96], # (2, 3550) 
            [-153.26, 64.53]] # (2, 3728)

min_lon = centers[location][0] - .65
max_lon = centers[location][0] + .65
min_lat = centers[location][1] - .25
max_lat = centers[location][1] + .25

box_lon = np.array([min_lon, max_lon, max_lon, min_lon, min_lon])
box_lat = np.array([min_lat, min_lat, max_lat, max_lat, min_lat])

print(max(box_lon), min(box_lon), min_lon, max_lon, min_lat, max_lat)

IndexError: list index out of range

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.set_xlim(min_lon - .1, max_lon + .1)
ax.set_ylim(min_lat - .1, max_lat + .1)

In [ ]:
''' Get data '''
ds = xr.open_dataset("/Users/joshuamiller/Documents/SULI 2023/Data/ABoVE_Snow_Cover/data/Alaska_snow_extent_depth_2001-2017.nc4")

print("---------------------------------------------------")
print(ds.coords, " |", ds.prod, " |", ds.attrs)
print("---------------------------------------------------")
# Create Dask arrays for lat and lon
lat = da.from_array(ds["lat"].values, chunks=(1000, 1000))
lon = da.from_array(ds["lon"].values, chunks=(1000, 1000))
time = da.from_array(ds["time"].values, chunks=(365))

In [ ]:
print(type(lat))
#============================================================================================
# Use Dask to filter the lat and lon arrays
and_idx = np.array(np.where(np.logical_and(np.logical_and(lat >= min_lat, lat <= max_lat), 
                                           np.logical_and(lon >= min_lon, lon <= max_lon))))
#============================================================================================
# Compute the result and convert to numpy

time = time.compute()

print(str(time[0])[0:9], np.shape(time))

In [ ]:
print(np.shape(and_idx))

In [ ]:
print("lat.shape=", np.shape(lat), "lon.shape=", np.shape(lon), "time.shape=", time.shape[0])

In [ ]:
start_date = datetime.strptime('2001-1-5', '%Y-%m-%d')

target_start = datetime.strptime('2015-12-29', '%Y-%m-%d')
start = (target_start - start_date).days

target_end   = datetime.strptime('2017-1-5', '%Y-%m-%d')
end = (target_end - start_date).days

curr_date = start_date + timedelta(days=10)

## Create a subset of the snow depth data inside a box

In [ ]:
''' Iterate through time and get snow values in the box '''
path_csv = ['/Users/joshuamiller/Documents/SULI 2023/Data/Alatna_Box/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Huslia_Box/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc1/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc2/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc3/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc4/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc5/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc6/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc7/Snow/RowsOfSnow',
            '/Users/joshuamiller/Documents/SULI 2023/Data/Loc8/Snow/RowsOfSnow']

In [ ]:
snow_list = []
time_list = []

In [ ]:
for t in range(np.shape(time)[0]):
    curr_date = datetime.strptime(str(time[t])[0:10], '%Y-%m-%d')
    
    if (((curr_date - target_start).days >= 0) and ((target_end - curr_date).days >= 0)):
        print("----------------------------------------------------------------------------")
        print("~~~~~~~~~~~~~~~~~~ ", t, ', date :', curr_date, "~~~~~~~~~~~~~~~~~~~~")
        print("----------------------------------------------------------------------------")
        
        new_snow = ds['snow_depth'][t, :, :]

        print(np.shape(new_snow))
        new_snow_darray = da.from_array(new_snow, chunks=(1000, 1000))
        print(type(new_snow_darray), np.shape(new_snow_darray))

        new_snow = []
        for i in range(np.shape(and_idx)[1]):
            val = new_snow_darray[and_idx[0][i]][and_idx[1][i]].compute().values

            if np.isnan(val):
                print(t, i, "val = ", val)
                val = 0
            new_snow.append(val)

            #print("snow =", new_snow_darray[and_idx[0][i]][and_idx[1][i]].compute().values)

        snow_list.append(new_snow)
        time_list.append(time[t])
#==========================================================================================
''' Extract lat and lon values in the box '''
new_lat = []
new_lon = []
for i in range(np.shape(and_idx)[1]):
    new_lat.append(lat[and_idx[0][i]][and_idx[1][i]].compute())
    new_lon.append(lon[and_idx[0][i]][and_idx[1][i]].compute())

    print("lat =", lat[and_idx[0][i]][and_idx[1][i]].compute(),
          ", lon=", lon[and_idx[0][i]][and_idx[1][i]].compute())

In [ ]:
print(min(new_lat), max(new_lat), min(new_lon), max(new_lon), len(new_lat), len(new_lon), np.shape(snow_list))
print('----')
print(min_lat, max_lat, min_lon, max_lon, np.shape(and_idx))

In [ ]:
new_snow_arr = np.asarray(snow_list)

## Save this subset of the data as a .nc file

In [ ]:
new_snow_files = ["/Users/joshuamiller/Documents/SULI 2023/Data/Alatna_Box/Snow/snow_Alanta_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Huslia_Box/Snow/snow_Huslia_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc1/Snow/snow_Loc1_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc2/Snow/snow_Loc2_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc3/Snow/snow_Loc3_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc4/Snow/snow_Loc4_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc5/Snow/snow_Loc5_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc6/Snow/snow_Loc6_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc7/Snow/snow_Loc7_subset_grid.nc",
                  "/Users/joshuamiller/Documents/SULI 2023/Data/Loc8/Snow/snow_Loc8_subset_grid.nc"]

In [ ]:
''' Create an nc file of the new_snow data '''
#ncfile.close()
ncfile = nc.Dataset(new_snow_files[location], 'w', format='NETCDF4')

# Create dimensions
ncfile.createDimension('time', np.shape(new_snow_arr)[0])
ncfile.createDimension('x', np.shape(new_snow_arr)[1])

# Create variables
lat_var = ncfile.createVariable('lat', 'f4', ('x',))
lon_var = ncfile.createVariable('lon', 'f4', ('x',))
time_var = ncfile.createVariable('time', 'i4', ('time',))
snow_var = ncfile.createVariable('snow_depth', 'f4', ('time','x',))

lat_var.setncattr('dimensions', 'x')
lon_var.setncattr('dimensions', 'x')
time_var.setncattr('dimensions', 'time')
snow_var.setncattr('dimensions', 'x' 'time')

# Assign values to variables
lat_var[:] = new_lat
lon_var[:] = new_lon
time_var[:] = time_list
snow_var[:] = new_snow_arr

# Add variable attributes
lat_var.units = 'degrees north'
lon_var.units = 'degrees east'
time_var.units = 'date'
snow_var.units = 'm'

# Close the file
ncfile.close()

In [ ]:
''' Open to make sure it worked '''
f = nc.Dataset(new_snow_files[location], "r")

# Print the dimensions of the file
print(f.dimensions.keys())

# Print the variables in the file
print(f.variables.keys())

new_nc_lat = f.variables['lat']
new_nc_lat = new_nc_lat[:]

new_nc_lon = f.variables['lon']
new_nc_lon = new_nc_lon[:]

new_nc_time = f.variables['time']
new_nc_time = new_nc_time[:]

new_nc_snow = f.variables['snow_depth']
new_nc_snow = new_nc_snow[:]

f.close()

In [ ]:
print("lat.shape=", np.shape(new_nc_lat), "lon.shape=", np.shape(new_nc_lon), "time.shape=", np.shape(new_nc_time), "snow=", np.shape(new_nc_snow))

In [ ]:
 ''' Get points for the snow plot and box '''
points = [Point(xy) for xy in zip(new_nc_lon, new_nc_lat)]
points_gdf = gpd.GeoDataFrame(geometry=points)


''' Corners of the box of interest '''
points_box = [Point(xy) for xy in zip(box_lon, box_lat)]
points_box_gdf = gpd.GeoDataFrame(geometry=points_box)

In [ ]:
date = 0

new_snow_gdf = gpd.GeoDataFrame(geometry=points).assign(data=new_nc_snow[date, :].ravel())

## Make the plot

In [ ]:
''' Plot world outline '''
coast = gpd.read_file('/Users/joshuamiller/Documents/SULI 2023/Environment/natural_earth_vector/10m_physical/ne_10m_coastline.shp')

ak = gpd.read_file("/Users/joshuamiller/Desktop/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")

In [ ]:
new_snow_norm = Normalize(vmin=0, vmax=max(new_nc_snow[date, :].ravel()))
new_snow_cmap = LinearSegmentedColormap.from_list('custom', ['blue', 'green'], N=200) # Higher N=more smooth

In [ ]:
ak.plot(ax=ax, color='white', edgecolor='black', linewidth=0.1, alpha=1, legend=True) # GOOD lots the map

plt.draw()

In [ ]:
new_snow_gdf.plot(ax=ax, column='data', cmap=new_snow_cmap, norm=new_snow_norm, markersize=5, alpha=1, legend=True)

plt.draw()

In [ ]:
points_box_gdf.plot(ax=ax, color='lime', edgecolor='lime', linewidth=.5, alpha=1)

plt.draw()